In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

conn = sqlite3.connect('example.db')
def rsq(sql):
    return pd.read_sql(sql, conn)

In [ ]:
-- Create a new table for dim_author with an author column
CREATE TABLE dim_author (
    author varchar(256)  NOT NULL
);

-- Insert authors 
INSERT INTO dim_author
SELECT DISTINCT author FROM dim_book_star;

-- Add a primary key 
ALTER TABLE dim_author ADD COLUMN author_id SERIAL PRIMARY KEY;

-- Output the new table
SELECT * FROM dim_author;

In [12]:
df = rsq("""
-- Output the new table
SELECT * FROM dim_author;
""")
df['author_id'] = df.index+1    
df.to_sql('dim_author', conn, if_exists='replace', index=False)

18

In [13]:
rsq("""
-- Output the new table
SELECT * FROM dim_author;
""")

,author,author_id
0,Agatha Christie,1
1,Aldous Huxley,2
2,Alice Walker,3
3,Alice Waters,4
4,Barack Obama,5
5,Beverly Cleary,6
6,Cecily von Ziegesar,7
7,Dan Brown,8
8,David Foster Wallace,9
9,F. Scott Fitzgerald,10


In [16]:
rsq("""SELECT dim_store_star.state, sum(sales_amount)
FROM fact_booksales
	-- Join to get book information
    JOIN dim_book_star on fact_booksales.book_id = dim_book_star.book_id
	-- Join to get store information
    JOIN dim_store_star on fact_booksales.store_id = dim_store_star.store_id
-- Get all books with in the novel genre
WHERE  
    dim_book_star.genre = 'novel'
-- Group results by state
GROUP BY
    dim_store_star.state;""")

,state,sum(sales_amount)
0,British Columbia,39006.0
1,California,72672.6
2,Florida,24670.8
3,Louisiana,21258.6
4,Massachusetts,57017.4
5,New Jersey,20816.4
6,New York,45652.2
7,Ontario,49605.6
8,Quebec,32564.4
9,Texas,41071.8


In [15]:
rsq("""-- Output each state and their total sales_amount
SELECT dim_store_star.state, SUM(sales_amount)
FROM fact_booksales
	-- Join to get book information
    JOIN dim_book_star on fact_booksales.book_id = dim_book_star.book_id
	-- Join to get store information
    JOIN dim_store_star on fact_booksales.store_id = dim_store_star.store_id
-- Get all books with in the novel genre
WHERE  
    dim_book_star.genre = 'novel'
-- Group results by state
GROUP BY
    dim_store_star.state;""")

,state,SUM(sales_amount)
0,British Columbia,39006.0
1,California,72672.6
2,Florida,24670.8
3,Louisiana,21258.6
4,Massachusetts,57017.4
5,New Jersey,20816.4
6,New York,45652.2
7,Ontario,49605.6
8,Quebec,32564.4
9,Texas,41071.8


In [28]:
conn = sqlite3.connect('snowflakes.db')

In [30]:

# # df = rsq("select * from fact_booksales")
df = pd.read_clipboard()
df.to_sql('dim_city_sf', conn, if_exists='append',index=False)

24

In [31]:
rsq("""-- Output each state and their total sales_amount
SELECT dim_state_sf.state, sum(sales_amount)
FROM fact_booksales
    -- Joins for the genre
    JOIN dim_book_sf on fact_booksales.book_id = dim_book_sf.book_id
    JOIN dim_genre_sf on dim_book_sf.genre_id = dim_genre_sf.genre_id
    -- Joins for the state 
    JOIN dim_store_sf on fact_booksales.store_id = dim_store_sf.store_id 
    JOIN dim_city_sf on dim_store_sf.city_id = dim_city_sf.city_id
	JOIN dim_state_sf on  dim_city_sf.state_id = dim_state_sf.state_id
-- Get all books with in the novel genre and group the results by state
WHERE  
    dim_genre_sf.genre = 'novel'
GROUP BY
   dim_state_sf.state;""")

,state,sum(sales_amount)
0,British Columbia,39006.0
1,California,72672.6
2,Florida,24670.8
3,Louisiana,21258.6
4,Massachusetts,57017.4
5,New Jersey,20816.4
6,New York,45652.2
7,Ontario,49605.6
8,Quebec,32564.4
9,Texas,41071.8


In [33]:
conn = sqlite3.connect('./example.db')
rsq("""-- Output records that need to be updated in the star schema
SELECT * FROM dim_store_star
WHERE country != 'USA' AND country !='CA';""")

,store_id,store_address,city,state,country
0,798,23 Jeanne Ave,Montreal,Quebec,Canada
1,799,56 University St,Quebec City,Quebec,Canada
2,800,23 Verte Ave,Montreal,Quebec,Canada
3,801,33 Smith St,Toronto,Ontario,Canada
4,802,44 Green Blvd,Toronto,Ontario,Canada
5,803,55 Hover Lane,Ottawa,Ontario,Canada
6,804,68 Washington St,Vancouver,British Columbia,Canada
7,805,52 Kite St,Victoria,British Columbia,Canada
8,806,912 Main St,Vancouver,British Columbia,Canada
9,807,67 First Ave,Syracuse,New York,US


In [ ]:
-- Add a continent_id column with default value of 1
ALTER TABLE dim_country_sf
ADD continent_id int NOT NULL DEFAULT(1);

-- Add the foreign key constraint
ALTER TABLE dim_country_sf ADD CONSTRAINT country_continent
   FOREIGN KEY (continent_id) REFERENCES dim_continent_sf(continent_id);
   
-- Output updated table
SELECT * FROM dim_country_sf;

In [39]:
conn = sqlite3.connect('./snowflakes.db')
rsq("""-- Add the foreign key constraint
SELECT * FROM dim_country_sf;
""")

,country_id,country,continent_id
0,1,Canada,1
1,2,USA,1


In [42]:
conn = sqlite3.connect('./example.db')
# rsq("""-- Add the foreign key constraint
# SELECT * FROM dim_country_sf;
# """)

# df = pd.read_clipboard()
# df.to_sql('customers', conn, if_exists='append',index=False)

4